# Assignment 3

# Setup in Neo4j

- Import fraud-detection-50.dump file into project
- Create new DMBS from dump
    - Note: Match password to display results in Jupyter Notebook
- Install APOC and GDS plugins
- Open DBMS in browser

# Connect to the neo4j server and user Neo4j GDS

In [ ]:
import pandas as pd
import os

from neo4j import GraphDatabase
from graphdatascience import GraphDataScience

# Make an instance of Neo4j driver and GDS

In [ ]:
NEO4J_URI = os.environ.get("NEO4J_URI", "bolt://localhost:7687") #Note: this is the default URI for the Neo4j Sandbox

NEO4J_AUTH = ( "neo4j",  "12345678") #Note: this is the default password for the Neo4j Sandbox

driver = GraphDatabase.driver(NEO4J_URI, auth=NEO4J_AUTH)

gds = GraphDataScience(NEO4J_URI, auth=NEO4J_AUTH)

# Create the SHARED_PII Relationship (Step 5)
```
MATCH ( c:Client )-[ :HAS_EMAIL |:HAS_PHONE |:HAS_SSN ] -> (n) <-
[ :HAS_EMAIL |:HAS_PHONE |:HAS_SSN ]- ( d:Client )
WHERE id(c) < id(d)
WITH c, d, count(*) AS cnt
MERGE (c) - [ :SHARED_PII { count: cnt } ] -> (d)
```

# Create the projection graph (Step 6)
```
CALL gds.graph.project(
 'clientClusters' ,
 { Client: {
 label: 'Client' }
 },
 { SHARED_PII: {
 type: 'SHARED_PII',
orientation: 'UNDIRECTED',
properties: {
 count: {
 property: 'count' }
 }
 }
 }
 )
 YIELD graphName, nodeCount, relationshipCount
 ```

# Run the WCC Algorithm (Step 7)
```
CALL gds.wcc.stream(
  'clientClusters',
  {
    nodeLabels: ['Client'],
    relationshipTypes: ['SHARED_PII'],
    consecutiveIds: true
  }
)
YIELD nodeId, componentId
RETURN gds.util.asNode(nodeId).id AS clientId, componentId AS clusterId
```

# Display results in Jupyter Notebook

In [ ]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "12345678")) #NOTE: change the password and bolt port to match your local setup

with driver.session() as session:
    result = session.run("CALL gds.wcc.stream('clientClusters', {nodeLabels: ['Client'], relationshipTypes: ['SHARED_PII'], consecutiveIds: true}) YIELD nodeId, componentId RETURN gds.util.asNode(nodeId).id AS clientId, componentId AS clusterId")
    df = pd.DataFrame([record.data() for record in result])
print(df)

# Identify Fraud Rings (Step 8)
```
CALL gds.wcc.stream(
  'clientClusters',
  {
    nodeLabels: ['Client'],
    relationshipTypes: ['SHARED_PII'],
    consecutiveIds: true
  }
)
YIELD nodeId, componentId
 WITH gds.util.asNode(nodeId) AS clientId , componentId AS clusterId
 WITH clusterId, collect(clientId.id) AS clients

 WITH clusterId, clients, size(clients) AS clusterSize WHERE clusterSize >= 2
 UNWIND clients AS client
 MATCH (c:Client) WHERE c.id = client
 SET c.secondPartyFraudRing = clusterId
 ```

# Create bipartiite graph (Step 9)
```
MATCH (c:Client) WHERE c.secondPartyFraudRing IS NOT NULL
WITH collect(c) as clients
MATCH (n) WHERE n:Email OR n:Phone OR n:SSN
WITH clients, collect(n) AS piis
WITH clients + piis AS nodes
MATCH (c:Client) -[:HAS_EMAIL | :HAS_PHONE | :HAS_SSN]->(p)
WHERE c.secondPartyFraudRing IS NOT NULL
WITH nodes, collect({source: c, target: p}) as relationships
CALL gds.graph.project.cypher(
  'similarity',
  "UNWIND $nodes as n RETURN id(n) AS id, labels(n) AS labels",
  "UNWIND $relationships as r RETURN id(r['source']) AS source, id(r['target']) AS target, 'HAS_PII' as type",
  { parameters: { nodes: nodes, relationships: relationships } }
)
YIELD graphName, nodeCount, relationshipCount
RETURN graphName, nodeCount, relationshipCount
```

# Run the Node Similarity Algorithm (Step 10)
```
CALL gds.nodeSimilarity.mutate(
  'similarity',
  {
    mutateProperty: 'jaccardScore',
    mutateRelationshipType: 'SIMILAR_TO',
    topK: 15
  }
)
YIELD nodesCompared, relationshipsWritten, computeMillis;
```


# Write similarity relationships to the graph (Step 11)
```
CALL gds.graph.writeRelationship(
  'similarity',
  'SIMILAR_TO',
  'jaccardScore'
)
```

# Degree centrality score (Step 12)
```
CALL gds.degree.write(
  'similarity',
  {
    nodeLabels: ['Client'],
    relationshipTypes: ['SIMILAR_TO'],
    relationshipWeightProperty: 'jaccardScore',
    writeProperty: 'secondPartyFraudScore'
  }
)
```

# Flag clients as fraudsters (Step 13)
```
MATCH (c:Client)
WHERE c.secondPartyFraudScore IS NOT NULL
WITH percentileCont(c.secondPartyFraudScore, 0.95) AS threshold
MATCH (c:Client)
WHERE c.secondPartyFraudScore > threshold
SET c:SecondPartyFraudster
```

# List clients with potential fraudster labels (Step 14)
```
MATCH (c:Client)
WHERE c:SecondPartyFraudster
RETURN c.name, c.id
```

# Display results in Jupyter Notebook

In [ ]:
with driver.session() as session:
    result = session.run("MATCH (c:Client) WHERE c:SecondPartyFraudster RETURN c.name, c.id")
    df = pd.DataFrame([record.data() for record in result])
print(df)